<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Overview</a></span></li></ul></li><li><span><a href="#Load-the-Data" data-toc-modified-id="Load-the-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load the Data</a></span></li><li><span><a href="#Model-the-Data-with-PLS-DA" data-toc-modified-id="Model-the-Data-with-PLS-DA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model the Data with PLS-DA</a></span><ul class="toc-item"><li><span><a href="#Training-a-Hard-Model" data-toc-modified-id="Training-a-Hard-Model-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Training a Hard Model</a></span></li><li><span><a href="#Training-a-Soft-Model" data-toc-modified-id="Training-a-Soft-Model-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Training a Soft Model</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Testing</a></span></li></ul></li><li><span><a href="#Optimizing-the-Classifier" data-toc-modified-id="Optimizing-the-Classifier-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Optimizing the Classifier</a></span></li></ul></div>

In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook

import imblearn
import sklearn

from sklearn.model_selection import GridSearchCV

import sys
sys.path.append('../../')
import chemometrics

import numpy as np
import pandas as pd

import watermark
%load_ext watermark

/home/nam/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/nam/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/nam/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and wil

Overview
--------
This is a simple example of using variants of PLS-DA to do some analysis.

In [2]:
%watermark -t -m -v --iversions

pandas    0.25.1
imblearn  0.5.0
numpy     1.21.4
json      2.0.9
sklearn   0.22.2.post1
watermark 2.0.2
12:18:42 

CPython 3.7.4
IPython 7.8.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-166-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


# Load the Data

In [3]:
# Let's load some data from the tests/ for this example
df = pd.read_csv('../tests/data/plsda3_train.csv')

In [4]:
# You can see that samples are rows, columns are different features
df

,Group,Class,Name,Li,B,Na,Mg,Al,K,Ca,...,Eu,Gd,Dy,Ho,Er,Tm,Yb,Lu,Pb,U
0,1,JPN1,jpn_001,0.001287,0.435363,2.567587,143.601117,0.009235,818.961080,36.075419,...,1.318100e-04,0.000629,0.000041,0.000108,0.000385,0.000182,2.953350e-04,0.000059,0.000182,0.000174
1,1,JPN1,jpn_002,0.001474,0.385210,4.595786,276.591018,0.084693,863.273852,50.704790,...,1.823270e-04,0.000592,0.000386,0.000051,0.000148,0.000021,3.067320e-06,0.000023,0.000443,0.000723
2,1,JPN1,jpn_003,0.000748,0.289601,5.806715,117.037380,0.119564,851.174760,46.020288,...,7.401510e-07,0.000717,0.000353,0.000192,0.000434,0.000194,5.446410e-05,0.000016,0.001359,0.000029
3,1,JPN1,jpn_004,0.000882,0.525801,0.554544,335.195531,0.388480,836.126629,45.437616,...,1.283760e-04,0.000481,0.000124,0.000041,0.000012,0.000180,2.635580e-04,0.000029,0.000948,0.000236
4,1,JPN1,jpn_005,0.001387,0.659031,3.102831,213.051823,0.106865,756.238004,39.155470,...,1.014040e-04,0.000464,0.000608,0.000031,0.000178,0.000102,4.990830e-05,0.000125,0.001060,0.000130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,1,THA1,tha_372,0.008112,0.515135,3.623354,227.360308,0.075525,830.443160,39.884393,...,1.861820e-04,0.000506,0.000387,0.000062,0.000193,0.000139,7.063200e-07,0.000063,0.001554,0.000056
341,1,THA1,tha_373,0.000198,0.554359,2.752819,397.151899,0.171857,879.746835,45.727848,...,2.826660e-05,0.000265,0.000556,0.000160,0.000157,0.000057,1.746930e-04,0.000183,0.000947,0.000180
342,1,THA1,tha_374,0.006398,0.729045,3.337994,385.826772,0.268925,938.976378,41.141732,...,4.206440e-06,0.000730,0.000662,0.000173,0.000183,0.000141,2.974180e-04,0.000193,0.003334,0.000208
343,1,THA1,tha_375,0.009893,0.457777,3.021736,364.285714,0.144162,978.571429,49.285714,...,1.831960e-04,0.000115,0.000521,0.000150,0.000148,0.000111,2.903890e-04,0.000099,0.000721,0.000079


In [5]:
raw_x = np.array(df.values[:,3:], dtype=float) # Extract features
raw_y = np.array(df['Class'].values, dtype=str) # Take the class as the target

# Model the Data with PLS-DA

In [6]:
from chemometrics.classifier.plsda import PLSDA

## Training a Hard Model

In [7]:
# Here the data are elemental levels so we will scale the X data
plsda = PLSDA(n_components=5, 
              alpha=0.05, 
              gamma=0.01, 
              not_assigned='UNKNOWN', 
              style="hard", 
              scale_x=True)

In [8]:
_ = plsda.fit(raw_x, raw_y)

In [ ]:
_ = plsda.visualize_2d(styles=['hard'])

In [ ]:
# We can see what samples are predicted to be using the predict() function.
pred = plsda.predict(raw_x)

In [ ]:
plsda.score(raw_x, raw_y)

In [ ]:
# The score() function is just tetsing how many are correctly predicted.  You can do this directly and 
# easily with the "hard" version of PLS-DA.
np.sum(np.array(pred).ravel() == raw_y) / raw_y.shape[0]

In [ ]:
# More complete figures of merit can be computed.
df, I, CSNS, CSPS, CEFF, TSNS, TSPS, TEFF = plsda.figures_of_merit(pred, raw_y)

In [ ]:
df # Each row is what the sample IS, each column is what the PREDICTION is.

In [ ]:
I # Total fo each category

In [ ]:
CSNS

In [ ]:
CSPS

In [ ]:
CEFF

In [ ]:
TSNS, TSPS, TEFF

## Training a Soft Model

In [ ]:
# Here the data are elemental levels so we will scale the X data
plsda = PLSDA(n_components=5, 
              alpha=0.05, 
              gamma=0.01, 
              not_assigned='UNKNOWN', 
              style="soft", 
              scale_x=True)

In [ ]:
_ = plsda.fit(raw_x, raw_y)

In [ ]:
# You can visualize both the hard and soft boundaries if you train a soft model.
# With a hard model, you only get the hard boundaries by default.
_ = plsda.visualize_2d(styles=['hard', 'soft'])

In [ ]:
# We can see what samples are predicted to be using the predict() function.
pred = plsda.predict(raw_x)

In [ ]:
# Samples can now be predicted to belong to multiple classes.
pred[:10]

In [ ]:
# More complete figures of merit can be computed.
df, I, CSNS, CSPS, CEFF, TSNS, TSPS, TEFF = plsda.figures_of_merit(pred, raw_y)

In [ ]:
df

## Testing

First, let's test on other pure samples that weren't in the training set.

In [ ]:
df = pd.read_csv('../tests/data/plsda3_test.csv')
raw_x_t = np.array(df.values[:,3:], dtype=float)
raw_y_t = np.array(['THA2']*len(raw_x), dtype=str)

In [ ]:
pred = plsda.predict(raw_x_t)
df, I, CSNS, CSPS, CEFF, TSNS, TSPS, TEFF = plsda.figures_of_merit(pred, raw_y_t)

In [ ]:
df # Most foreign samples were CORRECTLY identified as being unknown

# Optimizing the Classifier

Here we took alpha as a meaningful choice of type I error rate, but it could also be adjusted.  Moreover, we arbitrarily selected the number of PCs to use in the PLSDA model.  We can use scikit-learn's pipelines to automatically optimize hyperparameters like this.

In [ ]:
# Here I've use an imblearn pipeline, but you can also use scikit-learn's pipeline if you don't want to 
# do any class balancing.

pipeline = imblearn.pipeline.Pipeline(steps=[
    # Insert other preprocessing steps here...
    # ("smote", ScaledSMOTEENN(random_state=1)), # For example, class balancing
    ("plsda", PLSDA(n_components=5, 
                    alpha=0.05,
                    scale_x=True, 
                    not_assigned='UNKNOWN',
                    style='soft', 
                   )
    )
])

# Hyperparameters of pipeline steps are given in standard notation: step__parameter_name
param_grid = [{
    # 'smote__k_enn':[1, 2, 3],
    # 'smote__k_smote':[1, 3, 3],
    # 'smote__kind_sel_enn':['all', 'mode'],
    'plsda__n_components':np.arange(1, 20, 2),
    'plsda__alpha': [0.07, 0.05, 0.03, 0.01],
    #'plsda__scale_x':[True, False],
    #'plsda__style':['hard', 'soft'],
}]

gs = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    n_jobs=-1,
    cv=sklearn.model_selection.StratifiedKFold(n_splits=3, shuffle=True, random_state=0),
    error_score=0,
    refit=True
)

_ = gs.fit(raw_x, raw_y)

In [ ]:
# The best parameters found can be accessed like this:
gs.best_params_

In [ ]:
gs.best_score_ # The best score it recieved was...

In [ ]:
# You can see detailed CV results here
gs.cv_results_

In [ ]:
# For a 1D optimization you can easily visualize where the best value is:
# plt.errorbar(np.arange(1, 20, 2), gs.cv_results_['mean_test_score'], yerr=gs.cv_results_['std_test_score'])
# plt.xlabel('n_components')
# plt.ylabel('Mean Test Score (TEFF)')

In [ ]:
# scikit-learn finds the optimum over the range, however, you may wish to simply look at these results
# and use a smaller value, perhaps at an "elbow", and re-train a new model separately.

In [ ]:
# The refit=True (default) refits the model on the data in the end so you can use it directly.
gs.best_estimator_.predict(raw_x)

In [ ]:
# You can visualize the training results
gs.best_estimator_.named_steps['plsda'].visualize_2d(styles=['hard', 'soft'])

In [ ]:
# Train 
gs.best_estimator_.named_steps['plsda'].score(raw_x, raw_y) # The score being used here is TEFF

In [ ]:
pred = gs.best_estimator_.named_steps['plsda'].predict(raw_x)
df, I, CSNS, CSPS, CEFF, TSNS, TSPS, TEFF = plsda.figures_of_merit(pred, raw_y)

In [ ]:
pred[:20]

In [ ]:
df

In [ ]:
CSNS

In [ ]:
CSPS

In [ ]:
CEFF

In [ ]:
TSPS, TSNS, TEFF

In [ ]:
np.any(gs.best_estimator_.named_steps['plsda'].check_outliers())

In [ ]:
# Test
gs.best_estimator_.named_steps['plsda'].score(raw_x_t, raw_y_t) # The score being used here is TEFF

In [ ]:
pred = gs.best_estimator_.named_steps['plsda'].predict(raw_x_t)
df, I, CSNS, CSPS, CEFF, TSNS, TSPS, TEFF = plsda.figures_of_merit(pred, raw_y_t)

In [ ]:
pred[:20]

In [ ]:
df